In [ ]:
!wget -c http://vectors.nlpl.eu/repository/20/180.zip
!unzip 180.zip
!pip install pymorphy2

--2020-10-18 17:29:20--  http://vectors.nlpl.eu/repository/20/180.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

Archive:  180.zip
replace meta.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace model.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace README? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format("model.bin", binary=True)
model.init_sims(replace=True)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import  f1_score

In [ ]:
import string
import re

аугментация данных

In [ ]:
def augment_word2vec(sentence, model=model):
    sent_aug = []
    for word in sentence.split(' '):
      p = morph.parse(word)[0]
      tag = p.tag.POS
      word = p.normal_form + '_' + tag
      if word in model:
          for i in model.most_similar(positive=[word], topn=1):
              word_aug = i[0].split('_')
              sent_aug.append(word_aug[0])
    str_sent =' '.join(sent_aug)
    return str_sent

sent = "мама помыла окно"
augment_word2vec(sent, model=model)

'мама вымывать окошко'

In [ ]:
 def augment_word2vec_pos(sentence, model=model):
    sent_aug = []
    for word in sentence.split(' '):
      p = morph.parse(word)[0]
      tag = p.tag.POS
      word = p.normal_form + '_' + tag
      if word in model:
          for i in model.most_similar(positive=[word]):
              word_aug = i[0].split('_')
              if word_aug[1] == str(tag):
                  sent_aug.append(word_aug[0])
                  break

    str_sent =' '.join(sent_aug)
    return str_sent

sent = "мама помыла окно"
augment_word2vec_pos(sent, model=model)

'бабушка вымывать окошко'

**Блок 2: Регрессия; Medium Stories**

In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
# !unzip '/content/drive/My Drive/medium_stories.zip'

In [ ]:
# можно не использовать колонки с тегами
usecols = ["Title", "Subtitle", "Image", "Author", "Publication", "Year", "Month", "Day", "Reading_Time", "Claps", "url", "Author_url"]

df = pd.read_csv("Medium_Clean.csv", sep=",", usecols=usecols)[:50000]
df["Claps"] = df["Claps"].astype(int)
df.head()

df['title_sibtitle'] = df.Title + ' ' + df.Subtitle
# df = df.dropna(subset=['title_sibtitle'])
df = df.dropna()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df

,Title,Subtitle,Image,Author,Publication,Year,Month,Day,Reading_Time,Claps,url,Author_url,title_sibtitle
3,The Value Generated through AI led Mindfulness,A true product co-created by the Millennials,1,Emiko Sawaguchi,MILLENNIALSTIMES,2017,8,1,5,4,https://millennials-times.com/creativeflower-e...,https://millennials-times.com/@emikosawaguchi,The Value Generated through AI led Mindfulness...
4,"Os Melhores Links de JulhoCmeras Vintage, Goog...",mas pera a?,1,Giovani Ferreira,NEW ORDER,2017,8,1,3,56,https://medium.com/neworder/os-melhores-links-...,https://medium.com/@gferreira,"Os Melhores Links de JulhoCmeras Vintage, Goog..."
5,The Future of AI: Redefining How We Imagine,Original article posted on FICO blogs,1,Syed Sadat Nazrul,Towards Data Science,2017,8,1,6,103,https://towardsdatascience.com/the-future-of-a...,https://towardsdatascience.com/@sadatnazrul,The Future of AI: Redefining How We Imagine Or...
12,Serverless Prefabricated PaaS Accelerates Busi...,Serverless Computing accelerates time to,1,Hari Harikrishnan,The Cerebrus,2017,8,2,3,1,https://medium.com/cerebrus/from-rpc-to-server...,https://medium.com/@harikris,Serverless Prefabricated PaaS Accelerates Busi...
16,Chatbot2022630,Amie Chan 201781,0,Amie C.,Chatbots bible and More.,2017,8,2,1,2,https://medium.com/chatbots-bible-and-more/cha...,https://medium.com/@ameeee_c,Chatbot2022630 Amie Chan 201781
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49979,Why should you care about voice interfaces?,Hey! But I already use my fingers to swipe on ...,1,Kore,UX Collective,2018,4,1,3,200,https://uxdesign.cc/why-should-you-care-about-...,https://uxdesign.cc/@akshaykore,Why should you care about voice interfaces? He...
49982,Introducing Meeting Attender,"We spend a lot of time in meetings. In fact, m...",1,My Meeting Video,My Meeting Video,2018,4,1,2,0,https://medium.com/my-meeting-video/introducin...,https://medium.com/@mymeetingvideo,Introducing Meeting Attender We spend a lot of...
49987,The Choice Not to Think,"To think, or not to think, that is the question",1,Shane Saunderson,The Robot Magazine,2018,4,1,6,23,https://medium.com/the-robot-magazine/the-choi...,https://medium.com/@ArtificialShane,"The Choice Not to Think To think, or not to th..."
49991,Is it smart to have artificial intelligence?,We were in Brooklyn heading to a favorite Mexi...,0,Yves Mulkers,The Data Intelligence Connection,2018,4,1,2,0,https://medium.com/the-data-intelligence-conne...,https://medium.com/@YvesMulkers,Is it smart to have artificial intelligence? W...


конструирую дополнительные признаки (использовать их буду позже)

In [ ]:
df['text_len'] = df['title_sibtitle'].apply(len)
df['text_len_tokens'] = df['title_sibtitle'].apply(lambda x: len(x.split()))
df['number_of_caps'] = df['title_sibtitle'].apply(lambda x: len([ch for ch in x if ch.isupper()]))
df['number_of_nonalphanum'] = df['title_sibtitle'].apply(lambda x: len([ch for ch in x if not ch.isalnum()]))

токенизирую

In [ ]:
def my_tokenizer(text):
    text = text.lower()
    text = re.sub(r'[^A-Za-z0-9]+'," ",text)
    # text = text.split(' ')
    # print(text)
    return text

In [ ]:
df['title_sibtitle'] = df['title_sibtitle'].apply(my_tokenizer)

эксплоративный анализ данных

In [ ]:
print(df.shape)
print('качественные данные', '\n')
print('Author: ', len(df['Author'].unique()))
print('Publication: ', len(df['Publication'].unique()))

print('\n', 'количественные данные')
print('Year: ', len(df['Year'].unique()))
print('Month: ', len(df['Month'].unique()))
print('Month: ', len(df['Month'].unique()))
print('Reading_Time: ', len(df['Reading_Time'].unique()))
print('Claps max: ', df['Claps'].max())
print('Image: :', len(df['Image'].unique()))

(7990, 17)
качественные данные 

Author:  4297
Publication:  3176

 количественные данные
Year:  2
Month:  12
Month:  12
Reading_Time:  31
Claps max:  10300
Image: : 2


качественные данные кодирую с помощью one-hot encoding. Данных в столбце автор и Publication слишком много, и при кодировании таким способом получатся сильно разреженные вектора, поэтому эти столбцы я пропущу

In [ ]:
df.Image.value_counts(normalize=True)

1    0.626909
0    0.373091
Name: Image, dtype: float64

In [ ]:
df.Author.value_counts()

                     137
sumeet jannu         104
DEEP AERO DRONES      78
Joyce Ter Horst       69
C Gavilanes           57
                    ... 
Andrew Haisting        1
Jacob Heinricher       1
Hasibul Kabir          1
Florence Fermanis      1
Yatharth Ipsum         1
Name: Author, Length: 4297, dtype: int64

In [ ]:
df.Reading_Time.value_counts()

2     1815
3     1721
4     1248
1     1021
5      739
6      439
7      289
8      186
9      120
10     103
0      102
11      61
12      31
13      26
15      22
14      16
16       9
19       8
17       7
18       5
23       4
24       3
26       3
21       3
31       2
25       2
20       1
28       1
36       1
27       1
22       1
Name: Reading_Time, dtype: int64

In [ ]:
import matplotlib.pyplot as plt

df[[ "Image", "Year", "Month", "Day", "Reading_Time", "Claps"]].corr()

,Image,Year,Month,Day,Reading_Time,Claps
Image,1.000000,0.004682,-0.015544,-0.024095,0.101406,0.061517
Year,0.004682,1.000000,-0.869153,0.009992,-0.027275,-0.006035
Month,-0.015544,-0.869153,1.000000,-0.041548,0.010947,-0.002713
Day,-0.024095,0.009992,-0.041548,1.000000,0.030841,-0.002976
Reading_Time,0.101406,-0.027275,0.010947,0.030841,1.000000,0.193460
Claps,0.061517,-0.006035,-0.002713,-0.002976,0.193460,1.000000


из таблицы видно, что признаки reading_time и image больше прочих коррелируют с claps, поэтому далее я буду использовать только эти два признака

**векторизую данные** 

In [ ]:
from sklearn.model_selection import train_test_split
import statistics
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
df[["Year", "Month", "Day", "Reading_Time"]] = scaler.fit_transform(df[["Year", "Month", "Day", "Reading_Time"]])
df[["Year", "Month", "Day", "Reading_Time"]]

,Year,Month,Day,Reading_Time
3,-1.170262,0.451345,-1.646118,0.463800
4,-1.170262,0.451345,-1.646118,-0.262535
5,-1.170262,0.451345,-1.646118,0.826967
12,-1.170262,0.451345,-1.532969,-0.262535
16,-1.170262,0.451345,-1.532969,-0.988870
...,...,...,...,...
49979,0.854509,-0.673235,-1.646118,-0.262535
49982,0.854509,-0.673235,-1.646118,-0.625703
49987,0.854509,-0.673235,-1.646118,0.826967
49991,0.854509,-0.673235,-1.646118,-0.625703


In [ ]:
for col in ["Year", "Month", "Day", "Reading_Time"]:
  v = statistics.variance(df[col])
  m = df[col].mean()
  print(m, v)

-8.378223716245111e-15 1.0001251721116535
3.1095971299232012e-15 1.0001251721116535
-2.295479895720599e-17 1.0001251721116535
-8.180634587206692e-16 1.0001251721116533


In [ ]:
df[['text_len', 'number_of_nonalphanum', 'text_len_tokens', 'number_of_caps']] = scaler.fit_transform(df[['text_len', 'number_of_nonalphanum', 'text_len_tokens', 'number_of_caps']])

In [ ]:
for col in ['text_len', 'number_of_nonalphanum', 'text_len_tokens', 'number_of_caps']:
  v = statistics.variance(df[col])
  m = df[col].mean()
  print(m, v)

7.425571769207556e-17 1.0001251721116535
1.572653841390178e-16 1.0001251721116535
-7.878553879380022e-17 1.0001251721116537
-8.392674679269018e-18 1.0001251721116537


In [ ]:
x_train, x_test, Y_train, Y_test = train_test_split(df[[ 'title_sibtitle', "Image", "Reading_Time", 'text_len', 'number_of_nonalphanum', 'text_len_tokens', 'number_of_caps']], df['Claps'], test_size=0.3)

In [ ]:
count_vect = CountVectorizer()
X_train = count_vect.fit_transform(x_train['title_sibtitle'])
X_train = np.concatenate((X_train.toarray(), x_train[['Image', "Reading_Time"]].to_numpy()), axis=1)
X_test = count_vect.transform(x_test.title_sibtitle)
X_test = np.concatenate((X_test.toarray(), x_test[['Image', "Reading_Time"]].to_numpy()), axis=1)

In [ ]:
X_train_with_new_f = count_vect.fit_transform(x_train['title_sibtitle'])
X_train_with_new_f = np.concatenate((X_train_with_new_f.toarray(), x_train[['Image', 'Reading_Time', 'text_len', 'number_of_nonalphanum', 'text_len_tokens', 'number_of_caps']].to_numpy()), axis=1)
X_test_with_new_f = count_vect.transform(x_test.title_sibtitle)
X_test_with_new_f = np.concatenate((X_test_with_new_f.toarray(), x_test[['Image', 'Reading_Time', 'text_len', 'number_of_nonalphanum', 'text_len_tokens', 'number_of_caps']].to_numpy()), axis=1)

строю модели

In [ ]:
!pip install sklearn

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

In [ ]:
!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
def find_top_f(model, n):
  eli5_weights = eli5.formatters.as_dataframe.explain_weights_df(model)
  print(eli5_weights.shape)
  top_features = [int(i[1:]) for i in eli5_weights.feature if 'BIAS' not in i][:n]
  return top_features

# ridge

In [ ]:
ridge = Ridge(alpha=1)
model_ridge = ridge.fit(X_train, Y_train)
x_pred_ridge = model_ridge.predict(X_test)

In [ ]:
model_ridge_with_new_f = ridge.fit(X_train_with_new_f, Y_train)
x_pred_ridge_with_new_f = model_ridge_with_new_f.predict(X_test_with_new_f)

нахожу важные признаки


In [ ]:
top_features = find_top_f(model_ridge, 100)
X_train_eli5 = X_train[:,top_features]
X_test_eli5 = X_test[:,top_features]

ridge_eli5 = Ridge()
model_ridge_eli5 = ridge_eli5.fit(X_train_eli5, Y_train)
x_pred_ridge_eli5 = model_ridge_eli5.predict(X_test_eli5)

(19011, 3)


нахожу важные признаки с добавлением своих

In [ ]:
top_features = find_top_f(model_ridge_with_new_f, 100)
X_train_eli5 = X_train_with_new_f[:,top_features]
X_test_eli5 = X_test_with_new_f[:,top_features]

ridge_eli5 = Ridge()
model_ridge_eli5 = ridge_eli5.fit(X_train_eli5, Y_train)
x_pred_ridge_eli5_with_new_f = model_ridge_eli5.predict(X_test_eli5)

(19011, 3)


# lasso

In [ ]:
lasso = Lasso()
model_lasso = lasso.fit(X_train, Y_train)
x_pred_lasso = model_lasso.predict(X_test)

In [ ]:
model_lasso_with_new_f = lasso.fit(X_train_with_new_f, Y_train)
x_pred_lasso_with_new_f = model_lasso_with_new_f.predict(X_test_with_new_f)

нахожу важные признаки


In [ ]:
top_features = find_top_f(model_lasso, 70)
X_train_eli5 = X_train[:,top_features]
X_test_eli5 = X_test[:,top_features]

lasso_eli5 = Lasso()
model_lasso_eli5 = lasso_eli5.fit(X_train_eli5, Y_train)
x_pred_lasso_eli5 = model_lasso_eli5.predict(X_test_eli5)

(102, 3)


нахожу важные признаки с добавлением своих

In [ ]:
top_features = find_top_f(model_lasso_with_new_f, 70)
X_train_eli5 = X_train_with_new_f[:,top_features]
X_test_eli5 = X_test_with_new_f[:,top_features]

lasso_eli5 = Lasso()
model_lasso_eli5 = lasso_eli5.fit(X_train_eli5, Y_train)
x_pred_lasso_eli5_with_new_f = model_lasso_eli5.predict(X_test_eli5)

(102, 3)


In [ ]:
# linearregression = LinearRegression()
# model_linear = linearregression.fit(X_train, Y_train)
# x_pred_linear = model_linear.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
print('ridge, rmse:')
print(np.sqrt(mean_squared_error(Y_test, x_pred_ridge)))
print(np.sqrt(mean_squared_error(Y_test, x_pred_ridge_eli5)))
print(np.sqrt(mean_squared_error(Y_test, x_pred_ridge_with_new_f)))
print(np.sqrt(mean_squared_error(Y_test, x_pred_ridge_eli5_with_new_f)))
print()
print('lasso, rmse:')
print(np.sqrt(mean_squared_error(Y_test, x_pred_lasso)))
print(np.sqrt(mean_squared_error(Y_test, x_pred_lasso_eli5)))
print(np.sqrt(mean_squared_error(Y_test, x_pred_lasso_with_new_f)))
print(np.sqrt(mean_squared_error(Y_test, x_pred_lasso_eli5_with_new_f)))

# print(np.sqrt(mean_squared_error(Y_test, x_pred_linear)))
print()
print('ridge, r^2:')
print(r2_score(Y_test, x_pred_ridge))
print(r2_score(Y_test, x_pred_ridge_eli5))
print(r2_score(Y_test, x_pred_ridge_with_new_f))
print(r2_score(Y_test, x_pred_ridge_eli5_with_new_f))
print()
print('lasso, r^2:')
print(r2_score(Y_test, x_pred_lasso))
print(r2_score(Y_test, x_pred_lasso_eli5))
print(r2_score(Y_test, x_pred_lasso_with_new_f))
print(r2_score(Y_test, x_pred_lasso_eli5_with_new_f))
# print(r2_score(Y_test, x_pred_linear))

ridge, rmse:
424.66799180994667
329.2144629605109
424.45177357237714
329.2144629605109

lasso, rmse:
309.5579198945316
308.7077846775527
309.3496698531263
308.7077846775527

ridge, r^2:
-0.7289499398549988
-0.039061311267442944
-0.7271898103006424
-0.039061311267442944

lasso, r^2:
0.08131378166578651
0.08635280681514346
0.08254942823275879
0.08635280681514346


здесь первое значение - модель с признаками которой я ничего не делала

второе - выбираю и оставляю только важные признаки

третье - добавляю свои признаки к неотсортированным по важности признакам

четвертое - добавляю свои признаки и выбираю и оставляю только важные признаки

Из этих результатов видно, что lasso хорошо умеет выбирать важные признаки в отличие от ridge, именно поэтому качество модели ridge сильно улучшается при использовании ridge + eli5, а качество модели lasso не претерпевает сильных изменений 

Возможно, довольно низкое значение коэфициента детерминации что в первом, что во втором случае обусловленно недостатком более важных признаков, которых нет среди датасета и которые нельзя получить из имеющихся текстовых данных. Также можно было бы уменьшить размерность матрицы векторного представления текстов, чтобы матрица векторов не была разреженной.

лучший результат дает модель lasso + eli5 + new_features


**Grid Search**

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
lasso = Lasso()

parameters = {'alpha': [1, 3, 5], 'normalize':[False, True], 'positive':[False, True], 'selection':['cyclic', 'random']}
lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv = 3)
lasso_regressor.fit(X_train, Y_train)

print('Best score: {}'.format(lasso_regressor.best_score_))
print('Best parameters: {}'.format(lasso_regressor.best_params_))

Best score: -143597.1723015506
Best parameters: {'alpha': 1, 'normalize': False, 'positive': False, 'selection': 'cyclic'}


In [ ]:
model_lasso_best = lasso_regressor.best_estimator_.fit(X_train, Y_train)
model_lasso_best_pred = model_lasso_best.predict(X_test)
np.sqrt(mean_squared_error(Y_test, model_lasso_best_pred))

281.35402195120884

среднеквадратичная ошибка для модели lasso с наилучшими гиперпарамметрами


In [ ]:
ridge = Ridge()

parameters = {'alpha': [1, 3, 5], 'normalize':[False, True], 'solver': ['auto', 'svd']}
ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv = 3)
ridge_regressor.fit(X_train, Y_train)

print('Best score: {}'.format(ridge_regressor.best_score_))
print('Best parameters: {}'.format(ridge_regressor.best_params_))

Best score: -131734.22994299556
Best parameters: {'alpha': 5, 'normalize': True, 'solver': 'svd'}


In [ ]:
model_ridge_best = ridge_regressor.best_estimator_.fit(X_train, Y_train)
model_ridge_best_pred = model_ridge_best.predict(X_test)
np.sqrt(mean_squared_error(Y_test, model_ridge_best_pred))

348.7486330080029

среднеквадратичная ошибка для модели ridge с наилучшими гиперпарамметрами


**аугментация данных на примере первых трех строк фрейма**

In [ ]:
!wget -c http://vectors.nlpl.eu/repository/20/5.zip
!unzip 5.zip

--2020-10-18 11:57:43--  http://vectors.nlpl.eu/repository/20/5.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 574653290 (548M), 572306357 (546M) remaining [application/zip]
Saving to: ‘5.zip’

5.zip               100%[===================>] 548.03M  9.56MB/s    in 66s     

2020-10-18 11:58:51 (8.22 MB/s) - ‘5.zip’ saved [574653290/574653290]

Archive:  5.zip
replace meta.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: meta.json               
replace model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: model.bin               y

replace model.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: model.txt               y
y

replace README? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: README                  


In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
model = gensim.models.KeyedVectors.load_word2vec_format("model.bin", binary=True)
model.init_sims(replace=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
bat


In [ ]:
def augment_word2vec_pos(sentence, model=model):
    sent_aug = []
    for word in sentence.split(' '):
        normal_form = lemmatizer.lemmatize(word)
        if normal_form in model:
            for i in model.most_similar(positive=[normal_form], topn = 1):
                word_aug = i[0]
                sent_aug.append(word_aug)
        else:
            sent_aug.append(word)

    str_sent =' '.join(sent_aug)
    return str_sent

In [ ]:
df1 = pd.DataFrame(columns = ['aug_data', "Year", "Month", "Day", "Reading_Time"])
df1['aug_data'] = df['title_sibtitle'][:3].apply(augment_word2vec_pos)
df1[["Year", "Month", "Day", "Reading_Time"]] = df[["Year", "Month", "Day", "Reading_Time"]]

In [ ]:
df1

,aug_data,Year,Month,Day,Reading_Time
0,on-line animator mixamo vs norah AI on-line an...,2017,8,1,5
2,not-too-distant and AI unfriendly AI from Phys...,2017,8,1,3
3,of intrinsic generated through AI LED Mindfuln...,2017,8,1,5


In [ ]:
df[['title_sibtitle', "Year", "Month", "Day", "Reading_Time"]][:3]

,title_sibtitle,Year,Month,Day,Reading_Time
0,online animation mixamo vs norah ai online ani...,2017,8,1,5
2,futures of ai friendly ai from physics of the ...,2017,8,1,3
3,the value generated through ai led mindfulness...,2017,8,1,5


анализ результатов

низкое качество предсказания моделей может быть обусловленно недостатком признаков, плохой обработкой текста (я не удаляла стопслова, не лемматизировала текст т.д.), а также тем, что я не взяла все данные из исходного датасета, однако это у меня  не получилось сделать, потому что ОЗУ в колабе переполнялось и прога останавливалась еще на этапе векторизации.